### Import the required libraries 

In [ ]:
import odbind as odb
from odbind.survey import Survey
from odbind.well import Well
import pandas as pd
import numpy as np


In [ ]:
%load_ext autoreload
%autoreload 2

### Load the survey from Opendtect

In [ ]:
sdata = Survey("UnderGrad_Proj")

### Well Data

In [ ]:
wells = Well.names(sdata)

In [ ]:
wells

#### Well Logs informations

In [ ]:
well1 = Well(sdata,wells[0])


well1.log_info_dataframe()

In [ ]:
well1.track_dataframe()

#### Load all the well logs into a panda dataframe

In [ ]:
well_list = []

for well in wells:
    EB= Well(sdata, well)
    df = EB.logs_dataframe()[0]
    df['WELL'] = well
    well_list.append(df)    

In [ ]:
well_df = pd.concat(well_list)

In [ ]:
well_df.T

#### Names of all the available columns in the df

In [ ]:
hd = list(well_df.columns)

#### select columns to be used in the example

In [ ]:
sel_hd = ['dah','PEM2010_INPUT_DTC_ISO_1',
 'PEM2010_INPUT_NP_1',
 'PEM2010_INPUT_RHOB_1',
 'PEM2010_INPUT_RT_1',
 'PEM2010_INPUT_SWE_1',
 'PEM2010_INPUT_SWT_1',
 'PEM2010_INPUT_VCL_1'
       ,'WELL'  ]

In [ ]:
df = well_df[sel_hd]


In [ ]:
df.head()

In [ ]:
df.tail()

### Exploration Data Analysis

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
import seaborn as sn

In [ ]:
sn.pairplot(df.dropna().reset_index().drop(columns='dah'),hue='WELL' )

#### Automatic EDA using ydata-profiling

In [ ]:
from ydata_profiling.profile_report import ProfileReport
from ydata_profiling.compare_reports import compare

In [ ]:
ProfileReport(df)

#### drop null values 

In [ ]:
df_nan = df.dropna()

In [ ]:
pr1 = ProfileReport(df)

In [ ]:
pr2 = ProfileReport(df_nan)

In [ ]:
pr1.compare(pr2)

#### Remove outliers using PYOD

In [ ]:
from pyod.models import lof
from pyod.utils.ut

In [ ]:
lf = lof.LOF(contamination=0.01)

In [ ]:
lf.fit(df_nan)

In [ ]:
df_nan_an = df_nan.copy()

In [ ]:
df_nan_an['anomaly'] = lf.predict(df_nan)
df_nan_an

In [ ]:
fo2_1.put_log('Double GR', somelogs['dah']['m'].to_numpy(),\
              somelogs['Gamma Ray']['API'].to_numpy()*2,'API','GR',True]

In [ ]:
df_nan_an["scores"] = lf.decision_scores_
df_nan_an

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_nan_an["anomaly"].value_counts()

In [ ]:
inlier = df_nan[df_nan_an['anomaly']==0]
outlier = df_nan[df_nan_an['anomaly']==1]

In [ ]:
inlier

In [ ]:
outlier = outlier[["PEM2010_INPUT_DTC_ISO_1"]].sort_index()

In [ ]:
inlier["PEM2010_INPUT_DTC_ISO_1"].sort_index().plot()
plt.scatter(outlier.index,outlier["PEM2010_INPUT_DTC_ISO_1"],c='red' )

In [ ]:
df_inliers = inlier.copy()

#### Features and targets "in this examples we use SWE as the targets"

In [ ]:
df_inliers.columns

In [ ]:
df_inliers['RT_log'] = df_inliers[['PEM2010_INPUT_RT_1']].apply(np.log)

In [ ]:
df_inliers['RT_log'].sort_index().plot()

### Prepare data for Model Building

In [ ]:
Xdata = df_inliers.drop(columns=["PEM2010_INPUT_SWT_1","PEM2010_INPUT_SWE_1","dah"])
ydata = df_inliers[["PEM2010_INPUT_SWE_1"]]

#### split to training and test data

In [ ]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain, ytest = train_test_split(Xdata,ydata, test_size=0.25)


In [ ]:
xtrain